# gensim

Poznamy teraz inną bibliotekę do przetwarzania tekstu - gensim. 

Jest bardziej wydajna od nltk, pozwala przetwarzać duże zbiory dokumentów. 

Trzeba zainstalować gensim: https://radimrehurek.com/gensim/install.html


# Tworzenie reprezentacji wektorowej

Spróbujmy wykonać w gensim to czego się już nauczyliśmy, czyli transformować korpus do postaci wektorowej.

In [164]:
from gensim import corpora
import numpy as np

Tworzymy sobie zbiór dokumentów.

In [165]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]
np.savetxt('mycorpus.txt', documents, fmt='%s')

# Zad

Oczyszczamy tekst:

* tokenizacja

* usuwanie stopwords

* usuwanie tokenów występujących tylko 1 raz

* zmiana dużych liter na małe

Do powyższych zadań gensim nie ma dedykowanych narzędzi.

In [166]:
from pprint import pprint  # pretty-printer
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
          for document in documents]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


# Chcemy utworzyć reprezentację bag-of-words.

* Pakiet corpora posiada klasę Dictionary, która tworzy słownik z zadanych tokenów. 

* Odpowiada to sklearnowemu fit. 

* Słownik można zapisać na dysku i później go odzyskać.

In [167]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


Kluczami są tokeny a wartościami kolejne identyfikatory.

In [168]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


* Mając słownik możemy wziąć dowolny nowy dokument i przetransformować go. 

* To jest sklearnowy transform.

In [169]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec) 

[(0, 1), (1, 1)]


Funkcja doc2bow zlicza częstości wystąpienia kolejnych tokenów słownika w dokumencie. Zwraca wynik jako sparse vector.

# Zad
Prosze przetransformować "texts" do postaci wektorowej (nie ma do tego dedykowanej funkcji - trzeba użyć podejścia pythonowego)

In [170]:
corpus = [dictionary.doc2bow(text) for text in texts]
pprint(corpus)

corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]
[[(0, 1), (1, 1), (2, 1)], [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(2, 1), (5, 1), (7, 1), (8, 1)], [(1, 1), (5, 2), (8, 1)], [(3, 1), (6, 1), (7, 1)], [(9, 1)], [(9, 1), (10, 1)], [(9, 1), (10, 1), (11, 1)], [(4, 1), (10, 1), (11, 1)]]


Tą reprezentację również można zapisać na dysku.

In [171]:
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)  # store to disk, for later use

Korpusem w gensim jest macierz rzadka.

# Efektywne przetwarzanie tekstu

gensim pozwala operować na dużych zbiorach tekstu. Do pamięci nie jest wczytywany cały korpus a jedynie dokument po dokumencie.

Tworzymy klasę korpusu z funkcę __iter__, która ma dostęp do kolejnych tokenów korpusu. 

Postać tej funkcji może zostać dopasowana do jakiegokolwiek formatu naszego korpusu.

In [172]:
def __iter__(self):
        for line in open('mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

* Możemy stowrzyć obiekt korpusu. 

* Nie wczytuje to całego korpusu do pamięci. 

* W danym momencie tylko jeden dokument jest w pamięci.

In [173]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

Żeby zobaczyć co się kryje pod korpusem iterujemy po wektorach

In [174]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


Mimo że wynik nie różni się niczym od poprzedniego, to jest pamięciowo bardziej wydajny.

W podobny sposób możemy stworzyć słownik

In [175]:
from six import iteritems
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('mycorpus.txt'))
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


<h1>Kompatybilność z numpy</h1>

Możemy transformować zarówno gęste jak i rzadkie macierze